## Chaines使用
Chains 是一系列将各种组件链接在一起的工具，用于创建更复杂的应用程序。Chains 允许你将多个 LLM、提示模板、输出解析器和其他工具组合在一起，以创建可以执行特定任务的管道。

### 加载环境配置文件

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(".env")

print(os.getenv("OPENAI_API_KEY"))

### 开启链的日志功能
开启链的日志功能可以帮助你调试和理解链的执行过程。LangChain 提供了几种方法来启用日志记录，以便你可以查看链的内部操作。

#### 使用 langchain.globals 模块
LangChain 提供了一个全局配置模块 langchain.globals，你可以使用它来设置全局日志级别。

In [ ]:
import logging
import langchain.globals as globals

# 设置全局日志级别为 DEBUG
globals.set_debug(True)

# 或者，你可以使用 logging 模块设置更细粒度的日志级别
# logging.basicConfig(level=logging.DEBUG)

#### 使用环境变量
也可以使用环境变量来控制 LangChain 的日志记录。
- LANGCHAIN_DEBUG=true: 设置此环境变量会启用 LangChain 的调试模式。
    - 在 Linux/macOS 中：export LANGCHAIN_DEBUG=true
    - 在 Windows 中：set LANGCHAIN_DEBUG=true

In [ ]:
import os

os.environ["LANGCHAIN_DEBUG"] = "true"

print(os.getenv("LANGCHAIN_DEBUG"))

### LLMChain 基本的链
在 LangChain 0.3.21 版本中，LLMChain 已经被弃用，并且在后续版本中会被移除。
替代方案：使用 Runnable Sequence（| 运算符）
- LLMChain 是最基本的链，它将 LLM 与提示模板和可选的输出解析器结合在一起。
- 它接受用户输入，使用提示模板生成提示，将提示发送到 LLM，并（可选）使用输出解析器解析 LLM 的输出。

#### 使用 LLMChain
自 0.1.17 版起已弃用：改用。直到 langchain==1.0 才会将其删除。

In [ ]:
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

prompt_template = "帮我给{product}想三个可以注册的域名?"
# 创建 PromptTemplate 对象
prompt = PromptTemplate(
    input_variables=["product"], 
    template=prompt_template,
)

# 初始化 OpenAI 语言模型
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True, # 是否开启日志
)

result = chain.invoke({"product": "AI研习社"})
print(result)

#### 使用 Runnable Sequence
LangChain 推荐使用 Runnable Sequence（使用 | 运算符）来替代 LLMChain。  
以下是一个简单的示例，展示了如何使用 Runnable Sequence 替代 LLMChain：

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt_template = "帮我给{product}想三个可以注册的域名?"
# 创建 PromptTemplate 对象
prompt = PromptTemplate(
    input_variables=["product"], 
    template=prompt_template,
)

# 格式化提示模板，提供产品名称
# print(prompt.format(product="AI研习社"))

# 初始化 OpenAI 语言模型
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# 创建 LLM 链
# 创建了一个 LLM 链，将提示模板、OpenAI 语言模型和字符串输出解析器连接在一起
# | 运算符用于将链中的组件连接起来
# StrOutputParser() 创建一个字符串输出解析器实例，它将语言模型的输出转换为简单的字符串。
chain = prompt | llm | StrOutputParser()

# 调用 LLM 链
result = chain.invoke({"product": "AI研习社"})
print(result)

### SequentialChain 
SequentialChain 允许您将多个链按顺序连接在一起，并将一个链的输出作为下一个链的输入。

#### 使用 LLMChain 实现 SimpleSequentialChain 调用
- 简化操作：
  - SimpleSequentialChain 旨在简化顺序链的创建和执行。它适用于简单的链式操作，其中一个链的输出直接作为下一个链的输入。
- 隐式输入/输出：
  - SimpleSequentialChain 隐式地处理链之间的输入和输出传递。它假设每个链的输出都是一个字符串，并将其直接传递给下一个链。
- 限制：
  - 由于其简化性质，SimpleSequentialChain 在处理更复杂的链式操作时可能会受到限制。例如，它不支持自定义输入/输出变量或条件逻辑。
- 使用场景：
  - 适用于简单的文本处理流程，例如，将一个链生成的文本传递给另一个链进行摘要或翻译。
- 过时：
  - 在最新的langchain版本中，simpleSequentialChain 已经不推荐使用，官方更推荐使用Runnable接口来实现。


In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.sequential import SimpleSequentialChain

# 初始化 OpenAI 语言模型
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# 创建第一个 LLMChain
first_prompt = ChatPromptTemplate.from_template("帮我给{input}的公司起一个响亮容易记忆的名字?")
chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    output_key="output",
)

# 创建第二个 LLMChain
second_prompt = ChatPromptTemplate.from_template("用5个词来描述一下这个公司名字：{input}")
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    output_key="output",
)


# 创建 SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(
    # 链的列表，表示要按顺序执行的链。
    chains=[chain_one, chain_two],
    strip_outputs=False,
    verbose=True, # 打开日志
)

# 调用 SimpleSequentialChain
result = overall_simple_chain.invoke({"input": "AI教育培训机构"})
print(result)

# 执行流程：
# overall_simple_chain.invoke({"input": "AI教育培训机构"}) 将执行以下步骤：
# chain_one 将被执行，并使用 {"input": "AI教育培训机构"} 作为输入。
# chain_one 的输出将作为 chain_two 的输入。
# chain_two 将被执行，并使用 chain_one 的输出作为输入。
# chain_two 的输出将作为 overall_simple_chain.invoke() 的返回值。


#### 使用 Runnable 接口实现 SequentialChain 调用
- 灵活控制：
  - SequentialChain 提供了更灵活的控制，允许你明确指定每个链的输入和输出变量。
- 显式输入/输出：
  - 你需要显式地定义每个链的输入和输出变量，并在链之间传递数据。
- 功能强大：
  - SequentialChain 支持更复杂的链式操作，例如，使用不同的输入/输出变量、条件逻辑和自定义函数。
- 使用场景：
  - 适用于需要更精细控制链式操作的场景，例如，处理结构化数据、执行多个步骤的复杂工作流程或根据条件选择不同的链。
- 更新迭代：
  - SequentialChain，也在被Runnable接口逐步替换。

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.sequential import SimpleSequentialChain
from langchain_core.runnables import RunnablePassthrough # 用于在链中传递数据

# 初始化 OpenAI 语言模型
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# 使用 Runnable 接口创建第一个 Runnable 链
first_prompt = ChatPromptTemplate.from_template("帮我给{product}的公司起一个响亮容易记忆的名字?")
chain_one = first_prompt | llm | StrOutputParser()

# 使用 Runnable 接口创建第二个 Runnable 链
second_prompt = ChatPromptTemplate.from_template("用5个词来描述一下这个公司名字：{company_name}")
chain_two = second_prompt | llm | StrOutputParser()

# 创建 SimpleSequentialChain（使用 Runnable 接口）
# 使用字典和管道符 (|) 将两个链连接在一起，形成一个 Runnable 顺序链。
# `{"product": RunnablePassthrough()}`，用来将输入的product的值传递给第一个链。
# `{"company_name": RunnablePassthrough()}`，用来将第一个链的输出，传递给第二个链，并命名为company_name。
overall_chain = chain_one | chain_two

# 调用 SimpleSequentialChain
result = overall_chain.invoke("AI教育培训机构")

# 打印结果
print(result)

# 执行流程：
# overall_chain.invoke({"product": "AI学习平台"}) 将执行以下步骤：
# chain_one 将被执行，并使用 {"product": "AI学习平台"} 作为输入。
# chain_one 的输出将作为 chain_two 的输入。
# chain_two 将被执行，并使用 chain_one 的输出作为输入。
# chain_two 的输出将作为 overall_simple_chain.invoke() 的返回值。


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 初始化 OpenAI 语言模型
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# chain 1 任务：翻译成中文
first_prompt = ChatPromptTemplate.from_template("把下面内容翻译成中文:\n\n{content}")
chain_one = first_prompt | llm | StrOutputParser()

# chain 2 任务：对翻译后的中文进行总结摘要 input_key是上一个chain的output_key
second_prompt = ChatPromptTemplate.from_template("用一句话总结下面内容:\n\n{chinese_review}")
chain_two = second_prompt | llm | StrOutputParser()

# chain 3 任务:智能识别语言 input_key是上一个chain的output_key
third_prompt = ChatPromptTemplate.from_template("下面内容是什么语言:\n\n{chinese_summary}")
chain_three = third_prompt | llm | StrOutputParser()

# chain 4 任务:针对摘要使用指定语言进行评论 input_key是上一个chain的output_key   
fourth_prompt = ChatPromptTemplate.from_template("请使用指定的语言对以下内容进行回复:\n\n内容:{chinese_summary}\n\n语言:{language}")
chain_four = fourth_prompt | llm | StrOutputParser()

# overall 任务：翻译成中文->对翻译后的中文进行总结摘要->智能识别语言->针对摘要使用指定语言进行评论
overall_chain = (
    {"content": RunnablePassthrough()}  # 直接传递初始输入
    | chain_one  # 第一个链：翻译成中文
    | {"chinese_review": chain_one}  # 将 chain_one 的输出作为 chain_two 的输入
    | chain_two  # 第二个链：总结摘要
    | {"chinese_summary": chain_two}  # 将 chain_two 的输出作为 chain_three 的输入
    | chain_three  # 第三个链：识别语言
    | {"language": chain_three, "chinese_summary": chain_two}  # 将 chain_three 和 chain_two 的输出作为 chain_four 的输入
    | chain_four  # 第四个链：评论
)

content = "Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com)."
result = overall_chain.invoke(content)
print(result)

### RouterChain 
RouterChain 是一种特殊的链，它能够根据给定的输入动态地选择接下来要执行的链。简单来说，它就像一个“路由器”，根据输入的内容，将请求“路由”到不同的处理链。
- 主要功能：
  - 动态路由：
    - RouterChain 的核心功能是根据输入的内容，决定将请求发送到哪个链进行处理。
    - 这使得 LangChain 能够处理不同类型或不同意图的输入，并根据需要选择合适的处理流程。
  - 意图识别：
    - RouterChain 通常会使用一个“路由器链”，来识别用户输入的意图。
    - 这个路由器链会分析输入，并根据分析结果选择一个合适的“处理链”来处理请求。
  - 多链管理：
    - RouterChain 可以管理多个不同的处理链。
    - 每个处理链都负责处理特定类型或特定意图的输入。
  - 默认处理：
    - RouterChain 通常会有一个默认的处理链。
    - 当输入不符合任何已定义的处理链的条件时，它将被发送到默认处理链进行处理。
- 应用场景：
  - 智能客服：
    - RouterChain 可以用于构建智能客服系统，根据用户的问题类型（例如，订单查询、退款申请、产品咨询），将问题路由到不同的处理链。
  - 多功能助手：
    - RouterChain 可以用于构建多功能助手，根据用户的指令（例如，查询天气、播放音乐、发送邮件），将指令路由到不同的功能模块。
  - 复杂对话系统：
    - RouterChain 可以用于构建复杂的对话系统，根据对话的上下文和用户的意图，动态地调整对话流程。
- 工作原理：
  - 接收输入：
    - RouterChain 接收用户的输入。
  - 意图识别：
    - 路由器链分析输入，识别用户的意图。
  - 路由选择：
    - 根据意图识别的结果，RouterChain 选择一个合适的处理链。
  - 执行处理链：
    - 选定的处理链执行相应的处理逻辑，并生成输出。
  - 返回结果：
    - RouterChain 返回处理链的输出。

In [ ]:
from langchain_core.prompts import PromptTemplate

# 物理链
physics_template = """您是一位非常聪明的物理教授.\n
您擅长以简洁易懂的方式回答物理问题.\n
当您不知道问题答案的时候，您会坦率承认不知道.\n
下面是一个问题:
{input}"""
physics_prompt = PromptTemplate.from_template(physics_template)

# 数学链
math_template = """您是一位非常优秀的数学教授.\n
您擅长回答数学问题.\n
您之所以如此优秀，是因为您能够将困难问题分解成组成的部分，回答这些部分，然后将它们组合起来，回答更广泛的问题.\n
下面是一个问题:
{input}"""
math_prompt = PromptTemplate.from_template(math_template)

In [ ]:
from langchain.chains import ConversationChain
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

# 
prompt_infos = [
    {
        "name":"physics",
        "description":"擅长回答物理问题",
        "prompt_template":physics_template,
    },
    {
        "name":"math",
        "description":"擅长回答数学问题",
        "prompt_template":math_template,
    },
]

# 初始化 OpenAI 语言模型
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# 构建路由字典
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["input"]
    )
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
    )
    destination_chains[name] = chain

# 默认链
default_chain = ConversationChain(
    llm = llm,
    output_key="text"
)


In [ ]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router import MultiPromptChain

destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(MULTI_PROMPT_ROUTER_TEMPLATE)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(
    llm,
    router_prompt
)

In [ ]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [ ]:
chain.run("什么是牛顿第一定律?")

In [ ]:
chain.run("2+2等于几?")

In [ ]:
chain.run("两个黄鹂鸣翠柳，下一句?")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

# 初始化 LLM
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# 创建天气查询链
weather_template = """你是一个天气查询助手。请回答以下问题：
{input}"""
weather_prompt = PromptTemplate(template=weather_template, input_variables=["input"])
weather_chain = LLMChain(llm=llm, prompt=weather_prompt)

# 创建通用知识查询链
general_template = """你是一个通用知识问答助手。请回答以下问题：
{input}"""
general_prompt = PromptTemplate(template=general_template, input_variables=["input"])
general_chain = LLMChain(llm=llm, prompt=general_prompt)

# 创建路由器链的提示模板
router_template = """给定以下问题，请确定它属于哪个类别：
天气查询 或 通用知识查询。

问题：{input}
类别："""
router_prompt = PromptTemplate(template=router_template, input_variables=["input"])
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# 创建 RouterChain
destination_chains = {
    "天气查询": weather_chain,
    "通用知识查询": general_chain
}
 # 默认链
default_chain = general_chain 

chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain)

# 测试 RouterChain
print(chain.run("今天天气怎么样？"))
# print(chain.run("谁是阿尔伯特·爱因斯坦？"))

### transformation（转换链）
- 转换链（Transform Chains）  
Transform Chains 是一种 LangChain 链，专门用于对输入数据进行转换。
通过Transform Chains，您可以定义自定义的转换函数，并将它们集成到 LangChain 链中。
这使得您能够在数据发送给 LLM 之前，对其进行灵活的预处理。

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import (
    LLMChain,
    SimpleSequentialChain,
    TransformChain
)
from langchain_openai import ChatOpenAI

# 初始化 LLM
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

def transform_func(inputs:dict) -> dict:
    """
    转换函数
    它将输入文本按段落分割，并仅保留前三段，然后将它们重新连接成一个字符串。
    """
    text = inputs["text"]
    # 将文本按段落（以 \n\n 分隔）分割，并仅保留前三段。
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    return {"output_text":shortened_text}

# 创建文档转换链
transform_chain = TransformChain(
    # 指定输入变量的名称
    input_variables=["text"],
    # 指定输出变量的名称
    output_variables=["output_text"],
    # 指定转换函数
    transform=transform_func
)

template = """对下面的文字进行总结:
{output_text}

总结:"""
prompt = PromptTemplate(
    input_variables=["output_text"],
    template=template
)
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

# 创建顺序链
sequential_chain = SimpleSequentialChain(
    chains=[transform_chain, llm_chain],
    # 启用详细输出，以便在控制台中查看链的执行过程
    verbose=True
)

In [ ]:
with open("./sources/letter.txt") as f:
    letters = f.read()

In [ ]:
# print(letters)
sequential_chain.run(letters)

### 链的五种运行方式
在 LangChain 中，链（Chain）是将多个组件（如 LLM、提示模板、工具等）组合在一起形成一个整体工作流程的工具。LangChain 提供了多种方式来运行链，以满足不同的需求。以下是链的五种主要运行方式：

- run() 和 invoke() 用于运行单个输入。
- apply() 用于批量运行输入。
- stream() 用于流式传输输出。
- ainvoke(), astream(), 和 abatch() 用于异步运行。

In [ ]:
from langchain import (
    PromptTemplate,
    LLMChain
)
from langchain_openai import ChatOpenAI

# 初始化 LLM
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

prompt_template = "给做{product}的公司起一个名字?"
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template),
    verbose=True
)

# 调用方式一
# llm_chain("儿童玩具")

# 调用方式二
# llm_chain.run("儿童玩具")

# 调用方式三
# llm_chain.apply([
#    {"product":"儿童玩具"},
# ])

# 调用方式四
# llm_chain.generate([
#    {"product":"儿童玩具"},
# ])

# 调用方式五
llm_chain.predict(product="儿童玩具")